# Linear Regression
Well, why not.

Predicting blah blah blah from blah blah using linear regression with pytorch. Back to basics I guess. If I had a cuda core for everytime a tutorial started with this, I would be owning an Nvidia v100 by now. But then again, this exists for a reason. This is for me to come back in a few years to relearn stuff. Hello future me!

![alt text](https://drive.google.com/file/d/1K4Ks_eHx9KthxBOOKpgr5E08sBJXQ8nF/view?usp=sharing)

# By the way, ctrl+shift+space shows docstring in colab
unlike the Shift+Tab in jupyterlab/notebook and mouseover in kite vscode

In [0]:
import numpy as np
import torch

## Training Data

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Predicting yeild of apples and oranges
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Convert inputs and targets to pytorch tensors

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [4]:
# Random weights and biases
W = torch.randn(2, 3, requires_grad=True) # Transposed later
b = torch.randn(2, requires_grad=True)
W, b

(tensor([[ 0.3597, -0.6619, -0.4024],
         [ 1.9509,  0.4882,  0.3386]], requires_grad=True),
 tensor([-0.4954, -0.2067], requires_grad=True))

## Defining a model in Pytorch
Here, inputs x W^T + b = yeild

@ means matMul in pytorch

In [0]:
def model(x):
    return x @ W.t() + b # here, t() is a method for transpose

# Wait, what? That's it? WOW!

In [6]:
# dummy direct predictions
preds = model(inputs)
print(preds) # Wow, no wonder people say it is easier

tensor([[-35.8852, 189.4800],
        [-51.7603, 241.9596],
        [-81.2323, 254.5793],
        [ -7.1528, 232.3100],
        [-67.3841, 204.9757]], grad_fn=<AddBackward0>)


Wow, yeild of oranges is negative: Should we feed oranges to the tree or something? Anyway, let's train it then. Forget AI, it is oranges that is gonna take over the world.

## Loss Function

In [0]:
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff) / diff.numel() # numel() returns the number of elements

In [8]:
# compute loss
loss = mse(preds, targets); loss

tensor(19050.6230, grad_fn=<DivBackward0>)

## Compute Gradients
Compute the gradients of loss with respect to weigts and biases.

In [0]:
# Compute gradients of loss
loss.backward(retain_graph=True)
# Now the gradients are stored in the .grad property of respective tensors

In [10]:
print(W)
print(W.grad)

tensor([[ 0.3597, -0.6619, -0.4024],
        [ 1.9509,  0.4882,  0.3386]], requires_grad=True)
tensor([[-10187.8223, -12456.1592,  -7413.3486],
        [ 11596.1416,  10670.6445,   6891.0840]])


NOTE: Pytorch accumulates gradients. So the next time I call .backward(), the new gradient values will get added to the current ones. Hence we need to reset the gradients using .grad.zero_() as follows

In [11]:
W.grad.zero_()
b.grad.zero_()
W.grad, b.grad

(tensor([[0., 0., 0.],
         [0., 0., 0.]]), tensor([0., 0.]))

## Gradient Descent

Algorithm

1. Predict
2. Get loss
3. Compute gradient of loss w.r.t W and b
4. Adjust W and b by subtracting a small quantity proportional to the gradient
5. Reset grad to 0

In [12]:
# Generate prediction
loss = mse(preds, targets)
print(loss)
# Compute Gradients
loss.backward() # had to put retain_graph=True on the first backward() call to avoid error
print(W.grad)
print(b.grad)

tensor(19050.6230, grad_fn=<DivBackward0>)
tensor([[-10187.8223, -12456.1592,  -7413.3486],
        [ 11596.1416,  10670.6445,   6891.0840]])
tensor([-124.8829,  132.6609])


We should not track, calculate or modify gradients while updatinng weights. Hence add this in the scope of torch.no_grad(). After updating the weights, reset gradients to zero to prevent gradients from accumulating

In [0]:
eta = 1e-5 # learning rate
with torch.no_grad():
    W -= eta*W.grad
    b -= eta*b.grad
    W.grad.zero_()
    b.grad.zero_()

In [14]:
# New loss:
preds = model(inputs)
print('previous loss = ', loss)
loss = mse(preds, targets)
print('new loss = ',loss)

previous loss =  tensor(19050.6230, grad_fn=<DivBackward0>)
new loss =  tensor(13496.8125, grad_fn=<DivBackward0>)


## Training for 100 epochs

In [16]:
eta = 1e-5
for i in range(200):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        W -= eta*W.grad
        b -= eta*b.grad
        W.grad.zero_()
        b.grad.zero_()

preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(76.8011, grad_fn=<DivBackward0>)


Maybe they got a point that this is easier. My TF 1.13 mind is considered officially blown. Thank god for TF2, Thank god for eager execution. Im done with tf.Session()